# Проектирование признаков

In [78]:
import pandas as pd

df = pd.read_csv('data/wine_cleared.csv')
data = df.copy()
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            129971 non-null  object 
 4   points                 129971 non-null  int64  
 5   price                  129971 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               129971 non-null  object 
 8   taster_name            129971 non-null  object 
 9   taster_twitter_handle  129971 non-null  object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 12.9+ MB


None

### Разбор числовых величин

Например, в датасете винных обзоров числовым признаком является цена за бутылку вина. Вы можете округлить цену 35.363389$ в 35$, избавив данные от лишней информации.

Создадим новый признак price_round, означающий округлённую до целого числа цену за бутылку вина:

In [79]:
# для удобства сразу преобразуем признак в int
data['price_round'] = data['price'].round().astype(int)

Для таких преобразований важен контекст. В контексте вина для сомелье цена в 35$ и 35.363389$ одинакова. Вы делаете округление признака, чтобы модель также не сосредотачивалась на сотых. Это может помочь улучшить предсказание.

Однако такое преобразование неуместно в ситуациях, когда сотые важны. Например, при предсказании курса лиры стоимость валюты в 6.12 руб. и в 6.84 руб. — абсолютно разные ситуации. 

### Разбор текста

Текстовые признаки могут содержать в себе большое количество информации. Например, если бы наша модель умела распознавать текст, то из описания вина мы бы могли понять, какими словами сомелье чаще характеризуют качественное вино. Вы можете воспользоваться методами извлечения слов из текста, чтобы получить новые признаки.

В названии вина вы можете заметить указание года произведения вина. Чтобы узнать, влияет ли год на рейтинг вина, вам необходимо выделить год в отдельный признак.

Для того, чтобы извлечь из строки число, обозначающее год, необходимо воспользоваться регулярными выражениями.

Реализация такого механизма существует в pandas в работе со строками. Для того чтобы найти все числа в каждом значении серии, воспользуемся методом str.findall(). Метод возвращает все совпадения с заданным шаблоном в серии pandas. 

In [80]:
regex = '\d{4}' # регулярное выражение для нахождения чисел
data['year'] = data['title'].str.findall(regex).str.get(0)
data['year'] = data['year'].fillna(10000)
data['year'] = data['year'].apply(lambda x: int(x))
data['year'].value_counts()

2013    15853
2012    15723
2014    15568
2011    12531
2010    12149
        ...  
1503        1
1935        1
1969        1
1976        1
1945        1
Name: year, Length: 92, dtype: int64

Разберём регулярное выражение \d{4}:

* \d — класс символов, обозначает соответствие цифрам в диапазоне цифр [0-9];
* {4} в шаблоне означает искать четыре вхождения символа, указанного ранее. В нашем случае это будут четырехзначные числа.

Однако при поиске числа методом ``data['title'].str.findall(regex)`` результатом выполнения является список найденных цифр. Поэтому необходимо извлечь первый элемент из списка найденных методом ``str.get(0)``, где 0 — первый элемент в списке найденных чисел.

### Разбор категорий

В наборе данных винных обзоров самая популярная страна-производитель вина — США. Возможно, это не случайность, и факт производства в США влияет на рейтинг вина. Выделим этот факт.

Вы можете создать новый бинарный признак is_usa и присвоить ему 1 в случае, если вино произведено в США, иначе — 0.

In [81]:
data['is_usa'] = data['country'].apply(lambda x: 1 if x == 'US' else 0)

Задание 2.1
Выберите из списка две самых популярных (помимо США) страны, производящих вино.

In [82]:
data['country'].value_counts().nlargest(3).index[1:3]

Index(['France', 'Italy'], dtype='object')

Задание 2.2
Создайте бинарные признаки is_france, is_italy наподобие признака is_usa.

В ответ впишите результат выполнения кода ``data['is_france'].sum()`` и ``data['is_italy'].sum()``:

In [83]:
data['is_france'] = data['country'].apply(lambda x: 1 if x == 'France' else 0)
data['is_italy'] = data['country'].apply(lambda x: 1 if x == 'Italy' else 0)
france = data['is_france'].sum()
italy = data['is_italy'].sum()
print(f'Франция: {france} Италия: {italy}')

Франция: 22093 Италия: 19540


Задание 2.3
Создайте новый бинарный признак old_wine, где значение 1 будет обозначать, что вино старше 2010 года.

В ответ впишите результат выполнения кода ``data['old_wine'].sum()``:

In [84]:
data['old_wine'] = data['year'].apply(lambda x: 1 if x < 2010 else 0)
data['old_wine'].sum()

39781

Задание 2.7 (Самопроверка)
Создайте новый признак locality из признака title, который будет обозначать название долины/местности производства вина.

Например, в названии вина Rainstorm 2013 Pinot Gris (Willamette Valley) locality будет Willamette Valley. В названии Tandem 2011 Ars In Vitro Tempranillo-Merlot (Navarra) — Navarra.

Ответом на это задание является получившееся регулярное выражение и код преобразования.

In [94]:
regex_loc = '\(+(\w* +\w*)+\)' # регулярное выражение для нахождения чисел
data['locality'] = data['title'].str.findall(regex_loc).str.get(0)
data['locality'].value_counts().head(15)

 Valley              5906
Napa Valley          4480
Paso Robles          2350
Willamette Valley    1998
Finger Lakes         1564
Sonoma Coast         1474
 County              1381
Sonoma County        1246
Yakima Valley         997
 Provence             950
Colchagua Valley      830
 Highlands            809
Alexander Valley      784
Chianti Classico      740
Red Mountain          723
Name: locality, dtype: int64

## Работа с файлами

Чтобы проверить, влияет ли на качество вина населённость, выясним информацию о населении страны, в котором была произведена бутылка вина. 

Дата-инженеры предоставили нам файл country_population.zip (необходимо распаковать) с данными о населении по странам. 

Давайте прочтём его:

In [96]:
import pandas as pd
country_population = pd.read_csv('data/country_population.csv', sep=';')

country_population

,country,population
0,China,"1,411,778,724"
1,India,"1,386,584,581"
2,US,"333,022,386"
3,Indonesia,"271,350,000"
4,Pakistan,"225,200,000"
...,...,...
236,Niue,"1,549"
237,Tokelau,"1,501"
238,Vatican City,825
239,Cocos Islands,573
